In [126]:
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

from dotenv import load_dotenv
load_dotenv(override=True)

task_name = "whoami"

In [127]:
# autoryzacja

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token.")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token.


In [128]:
chat = ChatOpenAI(
    model='gpt-4'
)

hints = []

def try_to_guess(hints):

    new_hint = get_task(token)['hint']

    result = chat.invoke([
        SystemMessage(f"""We are playing "Guess Who I Am." The user is some character and gives you clues one by one, by which you can guess who it is about. 
                    If a given set of clues is enough and you know who the user is, then tell who they are. Just return the name of that person.
                    If the clues are not enough, then don't guess, but ask for another one by returning 0.
                    There are negative points for a wrong answer, so if you give an answer, make sure it is definitely correct.
                    
                    ### Previous hints:{hints}###"""),
        HumanMessage(f"New hint: {new_hint}"),
    ])

    return result.content, new_hint

In [129]:
if_guessed = 0

while(if_guessed == 0):
    answer, hint = try_to_guess(hints)

    if answer == '0':
        hints.append(hint)
    else:
        # wyślij odpowiedź
        response = send_answer(token, answer)

        if response:
            print("Odpowiedź z serwera:", response)
        else:
            print("Wystąpił błąd podczas wysyłania odpowiedzi.")
        hints.append(hint)
        if_guessed = 1    

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


Sprawdźmy o kogo chodziło i po czym model to zgadł:

In [130]:
print(hints)

['W momencie śmierci, jego majątek był szacowany na około 10,2 miliarda dolarów', 'często chodził boso po biurze']


In [131]:
print(answer)

Steve Jobs


GPT-4 zgadywał to po 2-3 wskazówkach, natomiast GPT-3.5 po 3-4 albo się mylił po wcześniejszych, więc dla 3.5 trzeba by dać dodatkowego prompta weryfikacyjnego.